In [2]:
import pyodbc
import matplotlib.pyplot as plt
import numpy as np
from sklearn import linear_model
from sklearn.model_selection import train_test_split
import sklearn.gaussian_process as gp
from meyerDB import cable_connection
# init db connection
conn = pyodbc.connect(cable_connection)
cursor = conn.cursor()


# get stuff
cursor.execute("SELECT project_id, count(*) FROM routed_cables GROUP BY project_id")
data = np.array(cursor.fetchall())
x = data[:, 0]
y = data[:, 1]
plt.bar(x, y)


<BarContainer object of 12 artists>